In [1]:
# Import Packages and create "Today" timestamp

import pandas as pd
import multiprocessing as mp
import shutil, re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from esri import get_df

today = datetime.now()

In [2]:
# Function used in the multiprocessor to webcrawl through different counties at one time

def webcrawl_loopnet(county):
#   Initialize and search based on county name
    print "Starting webcrawl for {}".format(county)
    property_data = []
    county_name = county.lower().replace(" ", "-").replace(",", "")
    driver = webdriver.Chrome()
    driver.set_window_position(0, 0)
    driver.set_window_size(0, 0)
    driver.minimize_window()
    loopnet_link = "https://www.loopnet.com/for-sale/{}/land/".format(county_name)
    driver.get(loopnet_link)
    paginator = driver.find_elements_by_tag_name("ol")
    max_page_str = paginator[0].text.split(" ")[-1]
    if max_page_str.isnumeric:
        max_page = int(max_page_str)
        loopnet_link = loopnet_link + "{}"
    else:
        return
    
#   Start the webcrawling action
    if max_page > 0:
        for x in range(max_page):
            driver.get(loopnet_link.format(x + 1))
            items = driver.find_elements_by_tag_name("article")
            for x in items:
                temp_data = []
                temp_sub_data = []
                temp_sf = []
                input_string = x.get_attribute("ng-click").split("'")[1].split("'")[0]
                temp_data.append(input_string)
                try:
                    link = input_string.split("Listing/")[1].split("/")[0]
                    if bool(re.search(r'\d', link)) and "upsell" not in link:
                        temp_data.append(" ".join(temp_data[0].split("Listing/")[1].split("/")[0].split("-")))
                        temp_data.append(x.find_elements_by_tag_name("a")[-1].text)
                        temp_data.append(county)
                        for index, val in enumerate(x.find_elements_by_tag_name("li")):
                            if "$" in val.text:
                                price = [val.text]
                            elif "SF" in val.text or "AC" in val.text:
                                numbers = re.compile('\d+(?:\.\d+)?')
                                temp_sf = [numbers.findall(val.text)[-1]] # Choose LAST number that is shown
                            elif val.text != "":
                                temp_sub_data.append(val.text)
                        temp_sf = [None] if not temp_sf else temp_sf
                        property_data.append(temp_data + price + temp_sf + [", ".join(temp_sub_data), today.strftime("%m/%d/%y")])
                except:
                    continue
    driver.quit()
    return property_data

In [3]:
# Actual execution of multiprocessor. For some reason, it's getting hung; work on this in the future

# counties_to_search = pd.read_csv("counties_to_search.csv")["County Name"].tolist()
# pool = mp.Pool(processes=4)
# results = [pool.apply_async(webcrawl_loopnet, args=(county,)) for county in counties_to_search]
# property_data = []
# for county in results:
#     property_data += county.get()

In [4]:
# Temporary solution while the multiprocessor is getting fixed; this will go one by one

all_property_data = []
counties_to_search = pd.read_csv("counties_to_search.csv")
counties_to_search = counties_to_search[counties_to_search["Active"] == True]["County Name"].tolist()
for county in counties_to_search:
    all_property_data += webcrawl_loopnet(county)

Starting webcrawl for Cook County, IL
Starting webcrawl for Lake County, IL
Starting webcrawl for Mchenry County, IL
Starting webcrawl for Kane County, IL
Starting webcrawl for DuPage County, IL
Starting webcrawl for Will County, IL


In [5]:
# Obtains previous raw_data.csv, combines with today's run, drops duplicates, and stores in raw_data

raw_data = pd.read_csv("raw_data.csv")
raw_data_columns = ["Link", "Address", "City-State", "County", "Price", "Size", "Details", "Upload"]
today_df = pd.DataFrame(all_property_data, columns = raw_data_columns)
raw_data = pd.concat([today_df, raw_data], ignore_index=True, sort=True)
raw_data = raw_data.drop_duplicates(subset="Link", keep='first', inplace=False)
raw_data = raw_data[raw_data_columns]

In [6]:
# Moves all of today's run into "Daily Import", moves previous raw_data file to archive, creates new raw_data file

today_df.to_csv("Daily Import/{}.csv".format(today.strftime("%Y-%m-%d %H.%M%p")), index=False)
shutil.move("raw_data.csv", "Raw Data/{}.csv".format(today.strftime("%Y-%m-%d %H.%M%p")))
raw_data.to_csv("raw_data.csv", index=False)

In [7]:
# Finds raw_data, filter out unwanted cities, combines data with master.csv and drops duplicates

df = pd.read_csv("raw_data.csv")

target_cities = pd.read_csv("cities_to_remove.csv")["City Name"].tolist()

filtered_df = df[~df["City-State"].str.contains("|".join(target_cities), na=False) & ~df["Address"].isna()]

master_df = pd.read_csv("master.csv")
final_df = pd.concat([master_df, filtered_df], ignore_index=True, sort=True)
final_df = final_df.drop_duplicates(subset="Link", keep='first', inplace=False)

In [8]:
# Adds additional demographic data as well as demographic score

for row in final_df.itertuples():
    if not(row.Inc1 >= 0) or not(row.Pop1 >= 0):
        try:
            esri_data = get_df(row.Address)
            
            final_df.loc[row.Index, ["Pop1"]] = esri_data[esri_data["Internal Name"] == "TOTPOP_CY"].iloc[0]["Ring 1"]
            final_df.loc[row.Index, ["Pop3"]] = esri_data[esri_data["Internal Name"] == "TOTPOP_CY"].iloc[0]["Ring 3"]
            final_df.loc[row.Index, ["Pop5"]] = esri_data[esri_data["Internal Name"] == "TOTPOP_CY"].iloc[0]["Ring 5"]

            final_df.loc[row.Index, ["Inc1"]] = esri_data[esri_data["Internal Name"] == "MEDHINC_CY"].iloc[0]["Ring 1"]
            final_df.loc[row.Index, ["Inc3"]] = esri_data[esri_data["Internal Name"] == "MEDHINC_CY"].iloc[0]["Ring 3"]
            final_df.loc[row.Index, ["Inc5"]] = esri_data[esri_data["Internal Name"] == "MEDHINC_CY"].iloc[0]["Ring 5"]
        except:
            continue

def apply_points(row):
    temp_points = 0
    if row["Pop1"] > 10000:
        temp_points += 1
    elif row["Pop1"] > 8000:
        temp_points += .5

    if row["Pop3"] > 35000:
        temp_points += 1
    elif row["Pop3"] > 28000:
        temp_points += .5

    if row["Inc1"] > 75000:
        temp_points += 1
    elif row["Inc1"] > 67500:
        temp_points += .5

    if row["Inc3"] > 75000:
        temp_points += 1
    elif row["Inc3"] > 67500:
        temp_points += .5
    
    return temp_points

def apply_hyperlink(row):
    return '=HYPERLINK("{}")'.format(row["Link"])

final_df["Points"] = final_df.apply(apply_points, axis=1)
final_df["Link_Click"] = final_df.apply(apply_hyperlink, axis=1)

ESRI Search Complete for 1166 McCarthy Rd Lemont IL
ESRI Search Complete for 29W632 US Highway 20 Elgin IL
ESRI Search Complete for 1236 1242 S Cicero Ave Cicero IL
ESRI Search Complete for 3810 183rd St Hazel Crest IL
ESRI Search Complete for 2800 W 159th St Markham IL
ESRI Search Complete for 4855 159th St Oak Forest IL
ESRI Search Complete for 1421 Joe Orr Rd Chicago Heights IL
ESRI Search Complete for 1120 1140 E Joe Orr Rd Lynwood IL
ESRI Search Complete for 1100 E Joe Orr Rd Lynwood IL
ESRI Search Complete for 221 N 1st Ave Maywood IL
ESRI Search Complete for 20248 S Cottage Grove Ave Chicago Heights IL
ESRI Search Complete for 17301 Lockwood Ave Oak Forest IL
ESRI Search Complete for 36939 N Green Bay Rd Waukegan IL
ESRI Search Complete for 3027 3028 Belvidere Waukegan IL
ESRI Search Complete for 1520 Prescott Dr Volo IL
ESRI Search Complete for 34736 N Il Route 83 Grayslake IL
ESRI Search Complete for 23379 State Rte 60 Round Lake IL
ESRI Search Complete for 20626 W Lake Cook R

In [9]:
# Archive master.csv and create new master.csv file

shutil.move("master.csv", "Master/{}.csv".format(today.strftime("%Y-%m-%d %H.%M%p")))
master_data_columns = ["Pop1", "Pop3", "Pop5", "Inc1", "Inc3", "Inc5", "Points", "Notes", "Link_Click"]
final_df = final_df[raw_data_columns + master_data_columns]
final_df = final_df.sort_values(by=["Points"], ascending=[False])
final_df.to_csv("master.csv", index=False)